# ➕ Extensión: Optimización de Consultas y Comparación

## 🎯 Objetivo

Complementar el laboratorio anterior evaluando el impacto de la caché, el broadcast join y Adaptive Query Execution (AQE) en una tarea práctica de análisis y agregación.

## 🧠 Técnicas de optimización incluidas
- Comparación de tiempo de ejecución con y sin caché
- Uso de `persist(StorageLevel.DISK_ONLY)` para pruebas controladas
- Control de broadcast join y AQE dinámicamente
- Análisis del plan de ejecución


In [0]:
# Importar StorageLevel para control de persistencia
from pyspark import StorageLevel
from pyspark.sql.functions import col
from time import time

# Crear dataset base
df_base = spark.range(0, 50_000_000).withColumn("mod", (col("id") % 1000))

## 🔁 Experimento A: Sin caché ni optimización
Medimos el tiempo de un `groupBy().count()` sin ninguna persistencia ni AQE.

In [0]:
start = time()
df_base.groupBy("mod").count().collect()
print(f"Tiempo sin caché ni AQE: {round(time() - start, 2)} segundos")

## ⚙️ Activar AQE y caché

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", True)
df_base.cache().count()

## 🔁 Experimento B: Con caché y AQE activado

In [0]:
start = time()
df_base.groupBy("mod").count().collect()
print(f"Tiempo con caché y AQE: {round(time() - start, 2)} segundos")

En escenarios de datasets moderados y clusters sin alta carga, técnicas como cache() o AQE pueden no mostrar mejoras inmediatas y, de hecho, introducir ligero overhead por su inicialización.
Sin embargo, su verdadero valor se evidencia cuando el dataset es consultado múltiples veces, o en pipelines complejos donde evitar recomputaciones es crítico.


## Objetivo
Demostrar cómo el uso de `cache()` y `Adaptive Query Execution` mejora el rendimiento cuando un DataFrame se reutiliza múltiples veces en operaciones pesadas.

In [0]:
# Generar un DataFrame con datos sintéticos de 100 millones de registros
from pyspark.sql.functions import rand
df = spark.range(0, 100_000_000).withColumn("value", rand())

## Caso A: Sin caché, sin AQE (tres agregaciones secuenciales)

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", False)
import time
start = time.time()
df.filter("value > 0.9").count()
df.groupBy((df.id % 100)).count().collect()
df.filter("value < 0.1").agg({"value": "avg"}).collect()
print(f" Tiempo total sin caché ni AQE: {round(time.time() - start, 2)} seg")

## Caso B: Con caché y AQE, para evitar recomputaciones y optimizar joins

In [0]:
df.cache().count()  # Forzar persistencia en memoria
spark.conf.set("spark.sql.adaptive.enabled", True)
start = time.time()
df.filter("value > 0.9").count()
df.groupBy((df.id % 100)).count().collect()
df.filter("value < 0.1").agg({"value": "avg"}).collect()
print(f" Tiempo total con caché y AQE: {round(time.time() - start, 2)} seg")

Con `cache()` y AQE activado, las operaciones que reutilizan el DataFrame se benefician al no recalcular el plan lógico desde cero ni volver a leer datos de disco.
Este ejemplo simula un patrón real de exploración de datos masiva en pipelines de análisis iterativo.

## ✅ Comparación final
- Revisa Spark UI para ver diferencias en el plan físico.
- Compara tiempo de ejecución y stages utilizados.
- Observa si hubo broadcast joins no deseados.

Este mini proyecto demuestra cómo pequeñas configuraciones pueden mejorar significativamente el rendimiento sin necesidad de escalar el cluster.